In [2]:
# Read CSV files from NY Times and covidtracking.com
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
covidtrack_to_load = "Resources/all-states-history 10-20-2020.csv"
NYtimes_to_load = "Resources/data from NYT 10-20-2020.csv"
state_list = "Resources/state list.csv"

# Read Purchasing File and store into Pandas data frame
covidtrack_data = pd.read_csv(covidtrack_to_load)
NYtimes_data = pd.read_csv(NYtimes_to_load)
state_list_data = pd.read_csv(state_list)
state_list_data["state"].value_counts()

North Carolina              1
New Jersey                  1
Indiana                     1
Utah                        1
Virginia                    1
Louisiana                   1
Vermont                     1
Washington                  1
Mississippi                 1
Oklahoma                    1
Wisconsin                   1
Idaho                       1
New York                    1
California                  1
Florida                     1
Illinois                    1
Minnesota                   1
Pennsylvania                1
Texas                       1
Wyoming                     1
Iowa                        1
Guam                        1
Arizona                     1
Alabama                     1
Missouri                    1
New Hampshire               1
Rhode Island                1
Georgia                     1
Hawaii                      1
Kentucky                    1
Kansas                      1
Arkansas                    1
New Mexico                  1
Nevada    

In [68]:
# reduce columns of covidtrack_data
reduced_covidtrack_data = pd.DataFrame(covidtrack_data, columns = ["date", "state", "negative", "negativeIncrease","positive","positiveIncrease","totalTestResults","totalTestResultsIncrease"])
reduced_covidtrack_data["date"]=pd.to_datetime(reduced_covidtrack_data["date"])
reduced_covidtrack_data

,date,state,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,2020-10-20,AK,534093.0,10090,12432.0,212,546525.0,10302
1,2020-10-20,AL,1112559.0,4731,174528.0,1043,1265575.0,5475
2,2020-10-20,AR,1137234.0,7110,100441.0,844,1231652.0,7738
3,2020-10-20,AS,1616.0,0,0.0,0,1616.0,0
4,2020-10-20,AZ,1419675.0,6571,232937.0,1040,1647345.0,7560
...,...,...,...,...,...,...,...,...
12928,2020-01-24,WA,0.0,0,0.0,0,0.0,0
12929,2020-01-23,MA,NaN,0,NaN,0,2.0,1
12930,2020-01-23,WA,0.0,0,0.0,0,0.0,0
12931,2020-01-22,MA,NaN,0,NaN,0,1.0,0


In [69]:
# rename columns of NYtimes_data
NYtimes_data["date"]=pd.to_datetime(NYtimes_data["date"])

In [70]:
new_data= pd.merge(NYtimes_data, state_list_data, on="state",how="left")
final_data = pd.merge(new_data, reduced_covidtrack_data,left_on=["date","initial"],right_on=["date","state"],how="left")
final_data['negative'] = final_data['negative'].fillna(0)
final_data['negativeIncrease'] = final_data['negativeIncrease'].fillna(0)
final_data['positive'] = final_data['positive'].fillna(0)
final_data['positiveIncrease'] = final_data['positiveIncrease'].fillna(0)
final_data['totalTestResults'] = final_data['totalTestResults'].fillna(0)
final_data['totalTestResultsIncrease'] = final_data['totalTestResultsIncrease'].fillna(0)
# final_data["total count"]=final_data["positive"]+final_data["negative"]
final_data

,date,state_x,fips,cases,deaths,initial,state_y,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,2020-01-21,Washington,53,1,0,WA,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-22,Washington,53,1,0,WA,WA,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-23,Washington,53,1,0,WA,WA,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-24,Illinois,17,1,0,IL,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-24,Washington,53,1,0,WA,WA,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12769,2020-10-20,Virginia,51,167754,3485,VA,VA,2254727.0,13055.0,167754.0,926.0,2411940.0,13829.0
12770,2020-10-20,Washington,53,103845,2380,WA,WA,2145675.0,13091.0,98661.0,460.0,2244336.0,13551.0
12771,2020-10-20,West Virginia,54,20519,408,WV,WV,671290.0,3593.0,20519.0,226.0,691809.0,3819.0
12772,2020-10-20,Wisconsin,55,187898,1660,WI,WI,1712883.0,9070.0,187828.0,4686.0,1900711.0,13756.0


In [75]:
renamed_df = final_data.rename(columns={"state_x":"States"})
                   
data_group_by = renamed_df.groupby(["States",renamed_df['date'].dt.month_name().str[:3]], sort="States")

Total_positive = data_group_by["positiveIncrease"].sum()
Total_negaitive = data_group_by["negativeIncrease"].sum()
Total_test = data_group_by["totalTestResultsIncrease"].sum()

df= pd.DataFrame({"Total positive":Total_positive,
                                  "Total negaitive":Total_negaitive,
                                  "Total test":Total_test})
df

Total positive  Total negaitive  Total test
States  date                                             
Alabama Apr           6038.0          73879.0     79917.0
        Aug          38335.0         257485.0    289359.0
        Jul          49678.0         233137.0    280879.0
        Jun          20142.0         168957.0    184836.0
        Mar            981.0           6288.0      7269.0
...                      ...              ...         ...
Wyoming Jun            584.0           8695.0      9279.0
        Mar            109.0           1999.0      2108.0
        May            344.0          14588.0     14932.0
        Oct           3578.0          13542.0     17120.0
        Sep           2106.0          22781.0     24887.0

[454 rows x 3 columns]

In [41]:
output_data = "Output/result data.csv"
df.to_csv(output_data)

TypeError: Expected tuple, got str